# Plot strain-specific model fits and song statistics

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

from aux import get_seg
from disp import set_plot

cc = np.concatenate

FPS = 30.03
DT = 1/FPS

COMBOS = [('NM91', 'ZH23'), ('CM07', 'CSTul'), ('N30', 'ZW109'), ('WTG',)]

KEYS = ['_'.join(COMBO).lower() for COMBO in COMBOS]
REG_KEY = 'ridge_alpha_10'

FSTRAIN = 'data/simple/strains.csv'
MCOMBOS = []
for COMBO in COMBOS:
    temp = [(pd.read_csv(FSTRAIN)['STRAIN'] == strain) for strain in COMBO]
    MCOMBOS.append(np.any(temp, axis=0))
ICOMBOS = [MCOMBO.nonzero()[0] for MCOMBO in MCOMBOS]

In [3]:
df_behav_c = pd.read_csv('data/simple/c_song_f_behav.csv')

n_tr = np.max(df_behav.ID) + 1

df_behav_

NameError: name 'df_behav' is not defined

# Frac trials singing and singing in each mode over time

In [ ]:
from scipy.ndimage import gaussian_filter1d as smooth

for ICOMBO, COMBO in zip(ICOMBOS, COMBOS):
    dfs_tr = [df_behav[df_behav.ID == i] for i in ICOMBO]
    tr_lens = [len(df_tr) for df_tr in dfs_tr]
    
    SMOOTH = int(round(10/DT))

    songs = np.nan * np.ones((len(dfs_tr), np.max(tr_lens)))
    for ctr, df_tr in enumerate(dfs_tr):
        song = np.zeros(len(df_tr), dtype=int)
        song[np.array(df_tr['S'], dtype=bool)] = 1
        song[np.array(df_tr['P'] | df_tr['F'], dtype=bool)] = 2
        songs[ctr, :len(df_tr)] = song

    frac_ongoing = np.mean(~np.isnan(songs), axis=0)

    fig, axs = plt.subplots(1, 2, figsize=(14, 4), tight_layout=True)
    axs[0].plot(DT*np.arange(songs.shape[1]), frac_ongoing, lw=2)

    frac_singing = smooth(np.mean(songs > 0, axis=0)/frac_ongoing, SMOOTH)

    axs[0].plot(DT*np.arange(songs.shape[1]), frac_singing, lw=2, c='k')
    axs[0].set_ylim(0, 1.1)
    axs[0].set_xlabel('Time (s)')
    # axs[0].set_ylabel('Fraction trials still going')
    axs[0].legend(['Frac trials still going', 'Frac singing'])

    frac_sine = smooth(np.mean(songs == 1, axis=0)/frac_ongoing, SMOOTH)
    frac_pulse = smooth(np.mean(songs == 2, axis=0)/frac_ongoing, SMOOTH)

    axs[1].plot(DT*np.arange(songs.shape[1]), frac_sine, lw=2, c='b')
    axs[1].plot(DT*np.arange(songs.shape[1]), frac_pulse, lw=2, c='r')
    axs[1].set_ylim(0, 1.1)

    axs[1].set_xlabel('Time (s)')
    axs[1].legend(['Sine', 'Pulse'])

    for ax in axs:
        ax.grid()
        set_plot(ax, title=f'{COMBO}', font_size=14)

# Avg female movement speed over time

In [4]:
from scipy.ndimage import gaussian_filter1d as smooth

for ICOMBO, COMBO in zip(ICOMBOS, COMBOS):
    dfs_tr = [df_behav[df_behav.ID == i] for i in ICOMBO]
    tr_lens = [len(df_tr) for df_tr in dfs_tr]
    SMOOTH = int(round(10/DT))

    mtns = np.nan * np.ones((len(dfs_tr), np.max(tr_lens)))
    for ctr, df_tr in enumerate(dfs_tr):
        mtn = np.array(np.sqrt(df_tr['FFV']**2 + df_tr['FLS']**2))
        mtns[ctr, :len(df_tr)] = mtn 

    # normalize motion
    # for ctr in range(len(mtns)):
    #     mtns[ctr, :] = mtns[ctr, :] / np.nanmean(mtns[ctr, :])

    frac_ongoing = np.mean(~np.isnan(songs), axis=0)

    mtn_mean = smooth(np.nanmean(mtns, axis=0), SMOOTH)
    mtn_mean /= mtn_mean.max()

    fig, ax = plt.subplots(1, 1, figsize=(7, 4), tight_layout=True)
    ax.plot(DT*np.arange(songs.shape[1]), frac_ongoing, lw=2)
    ax.plot(DT*np.arange(songs.shape[1]), mtn_mean, lw=2, c='k')
    ax.set_xlabel('Time (s)')
    ax.legend(['Frac trials still going', 'Avg female MTN'])

    ax.grid()
    set_plot(ax, y_lim=(0, 1.05), title=f'{COMBO}', font_size=14)

NameError: name 'df_behav' is not defined